In [62]:
from aocd.models import Puzzle


def parses(input):
    return [ (i,j) for i, line in enumerate(input.strip().split('\n'))
                   for j, c in enumerate(line)
                   if c == '#']
    

puzzle = Puzzle(year=2022, day=23)
data = parses(puzzle.input_data)

In [63]:
def visualize(pos):
    sx = int(min(z.real for z in pos))
    ex = int(max(z.real for z in pos))+1
    sy = int(min(z.imag for z in pos))
    ey = int(max(z.imag for z in pos))+1
    print(sx,ex,sy,ey)
    viz = ''
    for i in range(sx, ex):
        for j in range(sy, ey):
            viz += '#' if i+1j*j in pos else '.'
        viz += '\n'
    print(viz)
        

In [64]:
sample = parses("""
....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..
""")

In [65]:
# sample = parses(""".....
# ..##.
# ..#..
# .....
# ..##.
# .....
# """)

In [73]:
def add(a, b):
    return (a[0]+b[0], a[1]+b[1])

In [77]:
def simulate(data, steps):
    N = len(data)
    moves = [
        [(-1,0), [(-1,i) for i in (-1,0,1)]],
        [(1,0),  [(1,i) for i in (-1,0,1)]],
        [(0,-1), [(i,-1) for i in (-1,0,1)]],
        [(0,1),  [(i,1) for i in (-1,0,1)]],
    ]
    surroundings = set(sum([x for _, x in moves], start=[]))
    positions = set(data)
    steps = range(steps) if steps else itertools.count()
    
    for n in steps:
        tentative = {}
        for p in positions:
            tentative[p] = p
            if all( add(p,dp) not in positions for dp in surroundings):
                continue
            for i in range(4):
                move, condition = moves[(i+n)%4]
                if all( add(p,dp) not in positions for dp in condition):
                    tentative[p] = add(p, move)
                    break
        counts = Counter(tentative.values())
        new_positions = {p2 if counts[p2] == 1 else p 
                     for p, p2 in tentative.items()}
        if positions == new_positions:
            return n+1
        positions = new_positions
    return positions

In [78]:
simulate(sample, None)

20

In [79]:
%%time
simulate(data, None)

CPU times: user 10.3 s, sys: 79.6 ms, total: 10.4 s
Wall time: 10.4 s


1036

In [66]:
import itertools
from collections import Counter

def simulate(data, steps):
    N = len(data)
    moves = [
        (-1, [-1,-1+1j,-1-1j]),
        (1, [1,1+1j,1-1j]),
        (-1j, [-1j,1-1j,-1-1j]),
        (1j, [1j,1+1j,-1+1j]),
    ]
    surroundings = set(sum([x for _, x in moves], start=[]))
    positions = { i+j*1j for i,j in data}
    steps = range(steps) if steps else itertools.count()

    for n in steps:
        tentative = {}
        for p in positions:
            tentative[p] = p
            if all( (p+dp) not in positions for dp in surroundings):
                continue
            for i in range(4):
                move, condition = moves[(n+i) % 4]
                if all( (p+dp) not in positions for dp in condition ):
                    tentative[p] = p+move
                    break
                    
        counts = Counter(tentative.values())
        new_positions = {p2 if counts[p2] == 1 else p 
                     for p, p2 in tentative.items()}
        if positions == new_positions:
            return n+1
        positions = new_positions

    return positions        

In [67]:
def solve_a(data):
    positions = simulate(data, 10)
    sx = int(min(z.real for z in positions))
    ex = int(max(z.real for z in positions))+1
    sy = int(min(z.imag for z in positions))
    ey = int(max(z.imag for z in positions))+1
    return (ex-sx)*(ey-sy)-len(data)

In [68]:
solve_a(sample), solve_a(data)

(110, 4091)

In [69]:
def solve_b(data):
    return simulate(data, 0)

In [71]:
solve_b(sample)

20

In [72]:
%%time
solve_b(data)

CPU times: user 6.63 s, sys: 59.3 ms, total: 6.69 s
Wall time: 6.75 s


1036

In [124]:
from collections import Counter

In [125]:
def solve_a(data, rounds=10):
    all_ = [1,-1,1j,-1j,1+1j,1-1j,-1+1j,-1-1j]
    moves = [
        (-1, [-1,-1+1j,-1-1j]),
        (1, [1,1+1j,1-1j]),
        (-1j, [-1j,1-1j,-1-1j]),
        (1j, [1j,1+1j,-1+1j]),
    ]
    pos = data

    for _ in range(rounds):
        next_ = {}
        for x, d in pos.items():
            next_[x] = x
            if all(x+neigh not in pos for neigh in all_):
                continue
            for i in range(4):
                move, condition = moves[(d+i)%4]
                if all((x+neigh) not in pos for neigh in condition):
                    next_[x] = x+move
                    break
        counts = Counter(next_.values())
        new_pos = {}
        for x, y in next_.items():
            if counts[y] == 1:
                new_pos[y] = (pos[x] + 1) % 4
            else:
                new_pos[x] = (pos[x] + 1) % 4
        pos = new_pos
    
    sx = int(min(z.real for z in pos))
    ex = int(max(z.real for z in pos))+1
    sy = int(min(z.imag for z in pos))
    ey = int(max(z.imag for z in pos))+1
    return sum( i+j*1j not in pos for i, j in 
               itertools.product(range(sx,ex), range(sy,ey)) )

In [131]:
import itertools

In [134]:
def solve_b(data, rounds=10):
    all_ = [1,-1,1j,-1j,1+1j,1-1j,-1+1j,-1-1j]
    moves = [
        (-1, [-1,-1+1j,-1-1j]),
        (1, [1,1+1j,1-1j]),
        (-1j, [-1j,1-1j,-1-1j]),
        (1j, [1j,1+1j,-1+1j]),
    ]
    pos = data

    for n in itertools.count(1):
        next_ = {}
        for x, d in pos.items():
            next_[x] = x
            if all(x+neigh not in pos for neigh in all_):
                continue
            for i in range(4):
                move, condition = moves[(d+i)%4]
                if all((x+neigh) not in pos for neigh in condition):
                    next_[x] = x+move
                    break
        counts = Counter(next_.values())
        new_pos = {}
        for x, y in next_.items():
            if counts[y] == 1:
                new_pos[y] = (pos[x] + 1) % 4
            else:
                new_pos[x] = (pos[x] + 1) % 4
        if new_pos.keys() == pos.keys():
            return n
        pos = new_pos
    
    sx = int(min(z.real for z in pos))
    ex = int(max(z.real for z in pos))+1
    sy = int(min(z.imag for z in pos))
    ey = int(max(z.imag for z in pos))+1
    return sum( i+j*1j not in pos for i, j in 
               itertools.product(range(sx,ex), range(sy,ey)) )

In [135]:
solve_b(sample)

20

In [136]:
solve_b(data)

1036

In [126]:
solve_a(sample)

110

In [129]:
solve_a(data)

4091

In [118]:
visualize(pos)

0 6 0 5
..#..
....#
#....
....#
.....
..#..



In [4]:
def visualize(positions):

{(0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 6),
 (2, 0),
 (2, 4),
 (2, 6),
 (3, 1),
 (3, 5),
 (3, 6),
 (4, 0),
 (4, 2),
 (4, 3),
 (4, 4),
 (5, 0),
 (5, 1),
 (5, 3),
 (5, 5),
 (5, 6),
 (6, 1),
 (6, 4)}